In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Introduction

Name  : Ari Riscahyo Nugroho

Batch : 007

Github: Ari2109

This notebook contains Cyberbullying Tweet dataset from *J. Wang, K. Fu, C.T. Lu, “SOSNet: A Graph Convolutional Network Approach to Fine-Grained Cyberbullying Detection,” Proceedings of the 2020 IEEE International Conference on Big Data (IEEE BigData 2020), December 10-13, 2020*, available on https://www.kaggle.com/andrewmvd/cyberbullying-classification 

***TRIGGER WARNING!*** This notebook contains **offensive words regarding to age, gender, religion, ethnicity.** There might also be words containing **sexual harrassment.**

***PROCEED AT YOUR OWN RISK*** 

Before we continue, here are some notes regarding the dataset:

The author of the paper **used semi-supervised learning to generate and label some of the data in this dataset.** The goal for the paper is to **combat imbalanced dataset problems** occurring in NLP research. As such, **you might find incorrectly labeled data even in the original dataset**

**Humans' judgment are not bias-free. AI was built using input from humans' data.** **AI has limited capability**

Any seemingly incorrect labeled data could be caused from the AI failure to classify, creators' bias, readers' bias, or all of them. 


## Import Library

In [ ]:
#!pip install langdetect

In [ ]:
#!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import re
import nltk
from nltk.tokenize.casual import TweetTokenizer
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU, Dropout
from tensorflow.keras.models import Sequential

### Download Required NLTK Package

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

## Data Load and EDA

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Ari2109/data_repo/main/cyberbullying_tweets_2.csv")

In [ ]:
df.sample(10)

We'll check the what are the classes provided in this dataset

In [ ]:
df['cyberbullying_type'].unique()

We'll check if there's any null data in the dataset

In [ ]:
df.isnull().sum()

Seems like there isn't. Let's go to do some data cleaning

## Data Cleaning

We'll do with one sample text to make clear of what did we do to clean

### Sample Text Cleaning

In [ ]:
sample_text = df.loc[3]['tweet_text']
print(sample_text)

As this is twitter, we'll remove the @username. But we'll try to preserve the emoji/emoticon

In [ ]:
#deleting @'username' and punctuation, but keeping the emoji / emoticon 
sample_text = re.sub('@[^\s]+','',sample_text)
sample_text = re.sub('(?<=\w)[^\s\w](?![^\s\w])', '', sample_text)
print(sample_text)

We'll use TweetTokenizer to ensure that the emoticon stayed intact

In [ ]:
#word tokenizer, twitter style (keeping the emoticon in one place)
t = TweetTokenizer()
word_token = t.tokenize(sample_text)
print(word_token)

In [ ]:
fdist = FreqDist(word_token)
fdist.plot(30, cumulative=False)
plt.show()

Note that the author of the paper does *not* clean the stopword because the author believed that the stopwords might contain more pattern regarding to cyberbullying. As such, we also not do stopword-cleaning

We'll try to do Stemming and Lemmatization-with-POS here. We'll only use one of them in the end later

In [ ]:
# Lexicon Normalization
# Stemming

ps = PorterStemmer()

stemmed_words = []
for w in word_token:
    stemmed_words.append(ps.stem(w))

print("Filtered Sentence:", word_token)
print("Stemmed Sentence:", stemmed_words)

In [ ]:
# Lemmatization with POS

lem = WordNetLemmatizer()
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    lem.lemmatize(word,)
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_pos(tokenized_text):
    lemmatized = []
    for word in tokenized_text:
      lemmatized.append(lem.lemmatize(word, pos=get_wordnet_pos(word)))
    return lemmatized

lemmatized_token = lemmatize_pos(word_token)

def join_lemmatize_sentence(lemmatized_token):
  lemmatized_sentence = ""
  for x in lemmatized_token:
    lemmatized_sentence = lemmatized_sentence + " " + x
  lemmatized_sentence = lemmatized_sentence.strip()
  return lemmatized_sentence

join_lemmatize_sentence(lemmatized_token)
#print("Lemmatized with POS:", [lem.lemmatize(w, pos=get_wordnet_pos(w)) for w in word_token])

The decision is to use the lemmatization with POS in hope to keep some context in the texts

### Entire Dataset Cleaning

In [ ]:
df_prep = df.copy()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class proces_case_folding(BaseEstimator, TransformerMixin):
    
    def __init__(self, create = True):
        self.create = create
        
    def fit(self, X, y=None):
        return self
    
    def process(self, X):
        text = X.lower()
        #remove (at)username
        text = re.sub('@[^\s]+','',text)
        #remove RT flag
        text = re.sub('rt[\s]', '', text)
        #remove http and https
        text = re.sub('http[^\s]+', '', text)
        #remove & (occured in &amp and stuff)
        text = re.sub('&[^\s]+', '', text)
        text = re.sub(f'\d','', text)
        text = text.translate(str.maketrans(" "," ", string.punctuation))
        text = text.strip()
        return text
    
    def transform(self, X):
        X['Case Folding'] = X['tweet_text'].apply(self.process)
        return X['Case Folding']

In [ ]:
df_prep_e = df_prep.copy()
df_prep_e.head(1)

In [ ]:
##import zipfile
#archive = zipfile.ZipFile("../input/language-filtering.zip", 'r')
#model_pkl = archive.read('model_filter_language.pkl')

import joblib
load_model = joblib.load("../input/language-filtering/model_filter_language.pkl")

In [ ]:
df_prep_e['language'] = load_model.predict(df_prep_e)

In [ ]:
df_prep_e['language'].unique()

In [ ]:
drop_index = df_prep_e[df_prep_e['Case Folding'] ==''].index
print(drop_index.shape)
print(df_prep.shape)

In [ ]:
df_prep_e.groupby('language').count()['tweet_text']

In [ ]:
df_prep_e = df_prep_e[df_prep_e['language'] == 3]

In [ ]:
df_prep_e['language'].unique()

In [ ]:
df_prep_e.sample(5)

## Training - Evaluate Model

For the multiclass classification, we'll one-hot the target label before splitting them

In [ ]:
X = df_prep_e['Case Folding']
y = df_prep_e['cyberbullying_type']
y = pd.get_dummies(y).values
#print(pd.get_dummies(df_prep_e['cyberbullying_type']))

In [ ]:
from tensorflow.keras.layers import TextVectorization
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                    test_size=0.20, 
                                                    random_state=42)

In [ ]:
print(X_train.shape)
X_tensor = tf.data.Dataset.from_tensor_slices(X_train)

In [ ]:
# Mengatur tokenisasi menggunakan TextVectorization.

max_vocab_length = 30000 #untuk membatasi vocabnya sampai brp
max_length = 30 #maksimal kata dalam satu kalimat
shuffler = 5
batch_size = 32
text_vectorization = TextVectorization(max_tokens=max_vocab_length, #seberapa banyak token atau vocab
                                       standardize="lower_and_strip_punctuation", #auto menghilangkan tanda baca atau mengecilkan kata
                                       split="whitespace", #split berdasarkan spasi/jeda
                                       ngrams=None, #berapa kata yg ingin dijadikan vocab.
                                       output_mode='int',
                                       output_sequence_length=max_length)

In [ ]:
# Vektorisasi teks
text_vectorization.adapt(X_tensor)

In [ ]:
# Membuat layer embedding.

embedding = Embedding(input_dim=max_vocab_length, #batas kata untuk input (4000 tadi seperti di atas)
                      output_dim=max_length, #angka boleh berapapun
                      #embeddings_initializer="uniform",
                      input_length=max_length)

In [ ]:
# Membuat rancangan model untuk versi sequential API.

def get_basic_model(vectorization = text_vectorization):
    model = Sequential([
        vectorization,
        embedding,
        LSTM(128),
  #      Dropout(0.2),
 #       Dense(32, activation='relu'),
        Dense(6, activation='softmax')
])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

In [ ]:
train_seq = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_seq = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
test_seq = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
batches_train32 = train_seq.shuffle(5).batch(32)
batches_valid32 = valid_seq.shuffle(5).batch(32)
batches_test32 = test_seq.shuffle(5).batch(32)

batches_train16 = train_seq.shuffle(5).batch(16)
batches_valid16 = valid_seq.shuffle(5).batch(16)
batches_test16 = test_seq.shuffle(5).batch(16)

batches_train8 = train_seq.shuffle(5).batch(8)
batches_valid8 = valid_seq.shuffle(5).batch(8)
batches_test8 = test_seq.shuffle(5).batch(8)

batches_train64 = train_seq.shuffle(5).batch(64)
batches_valid64 = valid_seq.shuffle(5).batch(64)
batches_test64 = test_seq.shuffle(5).batch(64)

In [ ]:
model8 = get_basic_model()

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history_seq_8 = model8.fit(batches_train8, epochs=100, validation_data=batches_valid8, callbacks = [es_callback, cp_callback])

In [ ]:
hs8 = pd.DataFrame(history_seq_8.history)
hs8.plot()

In [ ]:
yp_seq = model8.predict(X_test)
y_pred = np.array(yp_seq).argmax(axis=1)
y_test_2 = np.argmax(y_test, axis=1)
report8L = pd.DataFrame(classification_report(y_test_2, y_pred, output_dict=True)).T
report8L.to_csv('8L.csv')
print(classification_report(y_test_2, y_pred))

In [ ]:
infer_text = '''
nice to meet you

'''

In [ ]:
def predict_cb(text, model_used):
    abc = pd.DataFrame(columns=['tweet_text'])
    abc.loc[0] = text
    lang = load_model.predict(abc)
    if(lang!=3):
        print('Sorry, this is not in English')
    else:
        print('This tweet is in English')
        pred = model_used.predict(abc['Case Folding'])
        pred = np.argmax(pred)
        if(pred == 0):
            print("This tweet is detected as cyberbullying. It is cyberbullying related to age")
        elif(pred==1):
            print("This tweet is detected as cyberbullying. It is cyberbullying related to ethnicity")
        elif(pred==2):
            print("This tweet is detected as cyberbullying. It is cyberbullying related to gender")
        elif(pred==3):
            print("This tweet is safe!")
        elif(pred==4):
            print("This tweet is detected as cyberbullying with no specific category.")
        elif(pred==5):
            print("This tweet is detected as cyberbullying. It is cyberbullying related to religion")
        print(abc)

In [ ]:
predict_cb(infer_text, model8)

In [ ]:
reconstruction_model = get_base_model()
reconstruction_model.load_weights(checkpoint_path)

In [ ]:
predict_cb(infer_text, reconstruction_model)

In [ ]:
yp_seq = reconstruction_model.predict(X_test)
y_pred = np.array(yp_seq).argmax(axis=1)
y_test_2 = np.argmax(y_test, axis=1)
report8L = pd.DataFrame(classification_report(y_test_2, y_pred, output_dict=True)).T
report8L.to_csv('recon8L.csv')
print(classification_report(y_test_2, y_pred))

In [ ]:
!zip -r modelcb.zip ./

In [2]:
pred = 9
if pred in range(0, 5):
    print('s')
else:
    print('ui')

ui


In [3]:
    list_out = {0 : "This tweet is detected as cyberbullying. It is cyberbullying related to age",
                1 : "This tweet is detected as cyberbullying. It is cyberbullying related to ethnicity",
                2 : "This tweet is detected as cyberbullying. It is cyberbullying related to gender",
                3 : "This tweet is safe!",
                4 : "This tweet is detected as cyberbullying with no specific category.",
                5 : "This tweet is detected as cyberbullying. It is cyberbullying related to religion"}

In [5]:
list_out[0]

'This tweet is detected as cyberbullying. It is cyberbullying related to age'